In [11]:
from diffusers import DiffusionPipeline
from transformers import CLIPTokenizer
import torch
torch_device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = (torch.bfloat16 if torch_device == "cuda" else torch.float32)
import numpy as np


In [2]:
tokenizer = CLIPTokenizer.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="tokenizer", torch_dtype=torch_dtype,)

In [105]:
x = "a inconceivably eating"
y = "a unimaginably eating"

words_x = x.split(' ')
words_y = y.split(' ')

max_len = tokenizer.model_max_length
mapper = np.zeros((max_len, max_len))

In [106]:
differing_indices = [i for i, (wx, wy) in enumerate(zip(words_x, words_y)) if wx != wy]
move_r, move_c = 0, 0
for idx in range(len(words_x)):
    if idx in differing_indices:
            tokens_x = tokenizer(words_x[idx], add_special_tokens=False, return_tensors="np")['input_ids'][0]
            tokens_y = tokenizer(words_y[idx], add_special_tokens=False, return_tensors="np")['input_ids'][0]
            len_x, len_y = len(tokens_x), len(tokens_y)
            move_c += (len_y-1)
            move_r += (len_x-1)
            if (len_y * len_x) == len_y or (len_y * len_x) == len_x:
                val = 1 / (len_y * len_x)
            else:
                val = 1 / len_y 
            for i in range(len_x):
                #val = 1 / (len_y)
                for j in range(len_y):
                        mapper[idx + i + 1][idx + j + 1] = val
                    
    else:
        mapper[idx + move_r + 1][idx + move_c + 1] = 1
mapper[0:6, 0:6]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.33333333, 0.33333333, 0.33333333,
        0.        ],
       [0.        , 0.        , 0.33333333, 0.33333333, 0.33333333,
        0.        ],
       [0.        , 0.        , 0.33333333, 0.33333333, 0.33333333,
        0.        ],
       [0.        , 0.        , 0.33333333, 0.33333333, 0.33333333,
        0.        ]])

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [12]:
max_len = tokenizer.model_max_length
tokens_x = tokenizer(text, add_special_tokens=False, padding=True, truncation=True, max_length=max_len, return_tensors="np")
tokens_y = tokenizer(text_2, add_special_tokens=False, padding=True, truncation=True, max_length=max_len, return_tensors="np")
max_len = tokenizer.model_max_length
mapper = np.zeros((max_len, max_len))

IndexError: index 328 is out of bounds for axis 0 with size 77

In [13]:
mapper.shape

(77, 77)

{'input_ids': array([[ 328,  720,  320, 5484, 2368]]), 'attention_mask': array([[1, 1, 1, 1, 1]])}